In [1]:
import pyvo as vo
import datetime as dt
from astropy.time import Time
from pypika import Table
import pandas as pd
import numpy as np


#NAXIS1 IS X IS RA
#NAXIS2 IS Y IS DEC

NotificationArea(sizing_mode='stretch_width')

In [2]:
url = "https://vo.astro.rug.nl/tap"
service = vo.dal.TAPService(url)

In [3]:
from pypika.queries import Query, QueryBuilder
from pypika.utils import builder, QueryException
from typing import Any, Optional
from enum import Enum

class Dialects(Enum):
    ADQL = 'adql'

class ADQLQuery(Query):
    @classmethod
    def _builder(cls, **kwargs: Any) -> "QueryBuilder":
        return ADQLQueryBuilder(**kwargs)

class ADQLQueryBuilder(QueryBuilder):
    QUOTE_CHAR = None
    ALIAS_QUOTE_CHAR = '"'
    QUERY_ALIAS_QUOTE_CHAR = ''
    QUERY_CLS = ADQLQuery

    def __init__(self, **kwargs: Any) -> None:
        super().__init__(dialect=Dialects.ADQL, **kwargs)
        self._top = None
        self._contains_point = None
        self._contains_box = None

    @builder
    def top(self, value):
        try:
            self._top = int(value)
        except ValueError:
            raise QueryException('TOP value must be an integer')

    def get_sql(self, *args, **kwargs):
        return super(ADQLQueryBuilder, self).get_sql(*args, groupby_alias=False, **kwargs)

    def _top_sql(self):
        if self._top:
            return 'TOP {} '.format(self._top)
        else:
            return ''

    @builder
    def contains_point(self, ra, dec, from_table, cone_radius):
        self._contains_point = [ra, dec, from_table, cone_radius]

    def _contains_point_sql(self):
        if self._contains_point:
            return "CONTAINS(POINT('ICRS', {}, {}), CIRCLE('ICRS', {}.ra, {}.dec, {}))=1 AND ".format(
                self._contains_point[0], self._contains_point[1], self._contains_point[2], self._contains_point[2], self._contains_point[3])
        else:
            return ''

    @builder
    def contains_box(self, ra, dec, from_table, width, height):
        self._contains_box = [ra, dec, from_table, width, height]

    def _contains_box_sql(self):
        if self._contains_box:
            return "CONTAINS(POINT('ICRS', {}, {}), BOX('ICRS', {}.ra, {}.dec, {}, {}))=1 AND ".format(
                self._contains_box[0], self._contains_box[1], self._contains_box[2], self._contains_box[2], self._contains_box[3], self._contains_box[4])
        else:
            return ''

    def _select_sql(self, **kwargs: Any) -> str:
        return "SELECT {distinct}{top}{select}".format(
            top=self._top_sql(),
            distinct="DISTINCT " if self._distinct else "",
            select=",".join(term.get_sql(with_alias=True, subquery=True, **kwargs) for term in self._selects)
        )

    @builder
    def _where_sql(self, quote_char: Optional[str] = None, **kwargs: Any) -> str:
        return " WHERE {contains_point}{contains_box}{where}".format(
            contains_point=self._contains_point_sql(),
            contains_box = self._contains_box_sql(),
            where=self._wheres.get_sql(quote_char=quote_char, subquery=True, **kwargs)
        )

In [8]:
def table_string():
    table_name = 'observations.raw'
    return table_name

def from_query():
    raw_observations = Table(table_string())
    return raw_observations

def gather_data(begin_date, end_date):
    begin_night_jd = Time(begin_date).jd
    end_night_jd = Time(end_date).jd
    table_name = from_query()
    query = ADQLQuery.from_(
        table_name
    ).select(
        table_name.filename, table_name.kw_IMAGETYP
    ).where(
        table_name.obs_jd[begin_night_jd:end_night_jd]
    ).top(10000)
    result = service.search(query.get_sql())
    data_pandas = result.to_table().to_pandas()
    return data_pandas

begin = dt.datetime(2009, 1, 1)
end = dt.datetime(2009, 12, 31)
df = gather_data(begin, end)
df

,filename,kw_IMAGETYP
0,/net/vega/data/users/observatory/images/090916...,Light Frame
1,/net/vega/data/users/observatory/images/090916...,Light Frame
2,/net/vega/data/users/observatory/images/090916...,Light Frame
3,/net/vega/data/users/observatory/images/090916...,Light Frame
4,/net/vega/data/users/observatory/images/090916...,Light Frame
...,...,...
5281,/net/vega/data/users/observatory/images/090524...,Dark Frame
5282,/net/vega/data/users/observatory/images/090524...,Dark Frame
5283,/net/vega/data/users/observatory/images/090524...,Light Frame
5284,/net/vega/data/users/observatory/images/090524...,Light Frame
